In [2]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In this notebook we generate and store random $k$-SAT instances for the experiments that we will be running.

Namely, for $n \in \{12, \dots, 20\}$ we generate $\{\underline{\sigma}\}_{i = 0}^{\nu - 1}, \underline{\sigma}_i \sim CNF(n, k, r)$ where $k = 8$ and $r = r_k$ an approximation of the satisfiability threshold.

In [ ]:
from benchmark.random_k_sat import RandomKSAT

k = 8
ns = [i for i in range(12, 21)]
v = 1000

# Generate
for n in ns:
	instances = RandomKSAT.from_poisson(n, k, instances=v, satisfiable=True)

	# Use correct directory
	parent_dir = os.path.dirname(os.getcwd())
	dir = f"{parent_dir}/benchmark/instances/n_{n}"

	# Write to files
	for i, formula in enumerate(instances.formulas):

		cnf_filename = f"{dir}/f_n{n}_k{k}_{i + 1100}.cnf"
		formula.to_file(cnf_filename)

In [6]:
from benchmark.random_k_sat import RandomKSAT

n12 = RandomKSAT.from_poisson(n=12, k=8, from_file=True).formulas[0]
n14 = RandomKSAT.from_poisson(n=14, k=8, from_file=True).formulas[0]

Calculating unsat counts:

In [ ]:
from multiprocessing.pool import Pool
import h5py 

from formula.cnf import CNF

k = 8
ns = [i for i in range(13, 21)]
v = 1100

def calc_and_write(fn: str):
	cnf = CNF.from_file(f'{fn}.cnf')
	counts = cnf.naive_counts
	with h5py.File(f'{fn}.hdf5', 'w') as file:
		file.create_dataset('counts', data=counts)

for n in ns:
	parent_dir = os.path.dirname(os.getcwd())
	dir = f"{parent_dir}/benchmark/instances/n_{n}"
	cnf_filenames = [f"{dir}/f_n{n}_k{k}_{i}" for i in range(v)]

	with Pool(processes=4) as pool:
		pool.map(calc_and_write, cnf_filenames)